# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [97]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [98]:
weather_data_df = pd.read_csv("../output_data/cities.csv")
weather_data_df.rename(columns={'Wind Speed':'Wind'},inplace=True)
weather_data_df.head()


,Unnamed: 0,City,Cloudiness,Date,Country,Humidity,Temperature,Latitude,Longtitude,Wind
0,0,victoria,74,1619290803,HK,86,298.15,22.2855,114.1577,0.45
1,1,zapolyarnyy,90,1619290906,RU,100,273.15,69.4154,30.8136,6.17
2,2,luanda,40,1619290853,AO,83,299.15,-8.8368,13.2343,1.54
3,3,bluff,100,1619290800,NZ,86,282.04,-46.6000,168.3333,2.24
4,4,adrar,6,1619291031,MR,11,305.73,20.5022,-10.0711,4.79


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [99]:
gmaps.configure(api_key=g_key)
coordinates = weather_data_df[['Latitude',"Longtitude"]]
humidity = weather_data_df['Humidity']
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(coordinates,weights=humidity)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [100]:
good_weather = pd.DataFrame(weather_data_df, columns = ["City","Cloudiness","Wind","Country","Temperature","Latitude","Longtitude"])
good_weather = good_weather[good_weather.Temperature<300]
good_weather = good_weather[good_weather.Temperature>294.2]
good_weather = good_weather[good_weather.Wind<10]
good_weather = good_weather[good_weather.Cloudiness==0]
good_weather.head()

,City,Cloudiness,Wind,Country,Temperature,Latitude,Longtitude
109,okhotsk,0,9.26,PE,296.15,-13.7000,-76.2167
246,sechura,0,2.57,SY,295.93,34.8890,35.8866
386,saquena,0,3.54,AO,299.39,-15.1961,12.1522
387,bubaque,0,3.90,ZW,297.00,-22.2167,30.0000
476,geresk,0,3.65,MZ,295.35,-23.8650,35.3833


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [114]:
hotels=[]

hotel_df=good_weather
hotel_df["Hotel Name"]=""

for x in range(len(good_weather)):
    lat = good_weather.iloc[x][5]
    long = good_weather.iloc[x][6]
    parameters = {
        "location":f"{lat},{long}",
        "radius": 5000,
        "types": "hotel",
        "key":g_key
    }
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url,params=parameters)
    json_response = response.json()
  
    hotels.append(json_response['results'][0]['name'])
hotel_df["Hotel Name"]= hotels
hotel_df.head(10)


,City,Cloudiness,Wind,Country,Temperature,Latitude,Longtitude,Hotel Name
109,okhotsk,0,9.26,PE,296.15,-13.7000,-76.2167,Pisco
246,sechura,0,2.57,SY,295.93,34.8890,35.8866,Tartus‎
386,saquena,0,3.54,AO,299.39,-15.1961,12.1522,Namibe
387,bubaque,0,3.90,ZW,297.00,-22.2167,30.0000,Beitbridge
476,geresk,0,3.65,MZ,295.35,-23.8650,35.3833,Maxixe


In [109]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longtitude"]]

In [129]:
# Add marker layer ontop of heat map
city_list=hotel_df["City"].tolist()
country_list=hotel_df["Country"].tolist()
hotel_list=hotel_df["Hotel Name"].tolist()
marker = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))